Import libraries and define directories for data

- tensorflow for image loading, layer formation & model compilation 
- numpy for matrix multiplication
- matplotlib for visualization

In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

# variables for directories (windows)
TEST_DIR = "C:/Users/Dowi/PycharmProjects/cats_vs_dogs/data/test/test1"
TRAIN_DIR = "C:/Users/Dowi/PycharmProjects/cats_vs_dogs/data/train/train"

Standardize images for processing

- Why? Images will 100% have different sizes (view /data/test/test1/)
- To counter that, initialize values for the image
- Create an NP array from the utils of TF 

In [ ]:
IMG_SIZE = (128, 128)

# load images from dir
def load_images(folder):
	images = []
	filenames = []

	for filename in os.listdir(folder):

		if filename.endswith('.jpg'):
			img_path = os.path.join(folder, filename)

			img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE)

			# normalize the data down to [0, 1], 255.0 = max RGB value
			img_array = tf.keras.utils.img_to_array(img) / 255.0
			images.append(img_array)
			filenames.append(filename)
	return (np.array(images), filenames)

train_images, train_filenames = load_images(TRAIN_DIR)

Get the labels of each image
- Cat will be 0
- Dog will be 1
- Unknown = -1

Assign the labels to train labels

CNN is written on model with relu activation and then a final sigmoid neuron

Compile and fit the model to train.

Use Matplotlib to display history

In [ ]:
# assign labels based on filename (assuming 'cat' or 'dog' in filename)
def get_label(filename):
	if 'cat' in filename.lower():
		return (0)
	elif 'dog' in filename.lower():
		return (1)
	else:
		return (-1)  # Unknown

# training labels determining model accuracy for backpropagation
train_labels = np.array([get_label(f) for f in train_filenames])

# convoluted neural network
model = tf.keras.Sequential([
	tf.keras.layers.Input(shape=(128, 128, 3)),
	tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
	tf.keras.layers.MaxPooling2D(2, 2),
	tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
	tf.keras.layers.MaxPooling2D(2, 2),
	tf.keras.layers.Flatten(),
	tf.keras.layers.Dense(64, activation='relu'),
	tf.keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train model
history = model.fit(train_images, train_labels, epochs=5, batch_size=32)

# plot training loss and accuracy
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['accuracy'], label='Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

For the testing images, same logic but do separate matplotlib display

In [ ]:
# Load test images
test_images, test_filenames = load_images(TEST_DIR)

# Predict using the trained model
preds = model.predict(test_images).flatten()
pred_labels = (preds > 0.5).astype(int)

# Get true labels from filenames
true_labels = np.array([get_label(f) for f in test_filenames])

# Display each image, prediction, and true label
for idx in range(len(test_images)):
	plt.imshow(test_images[idx])
	plt.axis('off')
	pred = 'Dog' if pred_labels[idx] == 1 else 'Cat'
	true = 'Dog' if true_labels[idx] == 1 else 'Cat'
	color = 'green' if pred_labels[idx] == true_labels[idx] else 'red'
	plt.title(f"Predicted: {pred}\nActual: {true}", color=color)
	plt.show()